In [43]:
import numpy as np
import pickle
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import functools
from sklearn.model_selection import train_test_split
import csv
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import tensorflow as tf

print ("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.0.0-alpha0
Eager execution: True


In [44]:
#value that the model is intended to predict.

LABEL_COLUMN = 'Smoke_Detector'
LABELS = [0, 1]

train_file_path='data1-sql.csv'
test_file_path='data1-revit.csv'

#Function to read the CSV data from the file and create a test dataset
def get_dataset_test(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=33,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset
#Function to read the CSV data from the file and create a train dataset
def get_dataset_train(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=330,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=100,
      ignore_errors=True,
      **kwargs)
  return dataset

#read the CSV data from the file and create a train dataset
raw_train_data = get_dataset_train(train_file_path)
raw_test_data = get_dataset_test(test_file_path)

#Column names in dataset
SELECT_COLUMNS = ['Fire_Sprinkler', 'Building_Occupancy', 'Room_Name', 'Room_Area', 'Smoke_Detector']

#read the CSV data from the file and create a train dataset
train_dataset = get_dataset_train(train_file_path, select_columns=SELECT_COLUMNS)
test_dataset = get_dataset_test(test_file_path, select_columns=SELECT_COLUMNS)


In [45]:
#Continuous data to be normalized.
train_batch, labels_batch = next(iter(train_dataset)) 
test_batch, labels_batch = next(iter(test_dataset)) 

class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_freatures = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

NUMERIC_FEATURES = ['Room_Area']

packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

train_batch, labels_batch = next(iter(packed_train_data))
test_batch, labels_batch = next(iter(packed_test_data))
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()


MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

array([[-0.23412707],
       [-0.7744204 ],
       [-0.7744204 ],
       [-0.23412707],
       [-0.7744204 ],
       [-0.23412707],
       [-0.23412707],
       [ 0.30616623],
       [-0.23412707],
       [ 0.30616623],
       [ 0.30616623],
       [ 1.9270462 ],
       [ 0.30616623],
       [ 0.30616623],
       [ 3.5479262 ],
       [-0.7744204 ],
       [-0.7744204 ],
       [ 4.0882196 ],
       [-0.7744204 ],
       [-0.23412707],
       [ 0.30616623],
       [-0.7744204 ],
       [-0.23412707],
       [ 0.30616623],
       [ 0.30616623],
       [ 0.30616623],
       [-0.23412707],
       [-0.23412707],
       [ 0.30616623],
       [ 0.30616623],
       [-0.7744204 ],
       [-0.7744204 ],
       [ 0.30616623],
       [ 0.30616623],
       [-0.7744204 ],
       [-0.23412707],
       [-0.7744204 ],
       [ 0.30616623],
       [-0.7744204 ],
       [-0.7744204 ],
       [-0.7744204 ],
       [-0.7744204 ],
       [ 1.386753  ],
       [ 4.0882196 ],
       [-0.7744204 ],
       [-0

In [46]:
#content in categories should be one of a limited set of options
CATEGORIES={}
CATEGORIES = {
     'Room_Name':[b'Bedroom'  ,
 b'Hall'  ,
 b'Living Room'  ,
 b'Closet'  ,
 b'Kitchen'  ,
 b'Master Bedroom'  ,
 b'Guest Bedroom'  ,
 b'Foyer'  ,
 b'Breakfast'  ,
 b'Den'  ,
 b'Office'  ,
 b'Playroom'  ,
 b'Corridor'  ,
 b'Elevator Lobby'  ,
 b'Lobby'  ,
 b'Electrical Room'  ,
 b'Mechanical Room'  ,
 b'Storage'  ,
 b'Garage'  ,
 b'Fire Pump Room'  ,
 b'Generator Room'  ,
 b'Gym'  ,
 b'Procedure Room'  ,
 b'Nurse Station'  ,
 b'Reception'  ,
 b'Exam Room'  ,
 b'Equipment Room'  ,
 b'MRI Room'  ,
 b'Control Room'  ,
 b'Operating Room'  ,
 b'Conference Room'  ,
 b'Break Room'  ,
 b'Bathroom'  ,
 b'Locker Room'  ,
 b'Nuc Med Room'  ,
 b'Spect CT Room'  ,
 b'Pre/Post Op'  ,
 b'Patient Bedroom'  ,
 b'Conference'  ,
 b'Breakroom'  ],
    'Building_Occupancy' : [b'Apartment Building',b'Medical Building',b'Office Building'],
    'Fire_Sprinkler' : [b'Yes',b'No']
}

#add any missing room names in database not specified in categories
df = pd.read_csv('data1-sql.csv')
for i in df['Room_Name']:
    if i not in CATEGORIES['Room_Name']:
        CATEGORIES['Room_Name'].append(i)
        
#create a collection
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [47]:
#Room Area was determined to not affect the prediction accuracy
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
#preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)


In [48]:
#Build the model
model = tf.keras.Sequential([
  categorical_layer,
  #preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),  
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [49]:
#model can be instantiated and trained
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data.shuffle(500)
model.fit(train_data, epochs=3)

#check its accuracy on the test_data set
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Epoch 1/3
100/100 [==============================] - 3s 35ms/step - loss: 0.2526 - accuracy: 0.7841
Epoch 2/3
100/100 [==============================] - 3s 29ms/step - loss: 0.0339 - accuracy: 0.9848
Epoch 3/3
100/100 [==============================] - 3s 32ms/step - loss: 0.0324 - accuracy: 0.9848
      1/Unknown - 0s 17ms/step - loss: 1.4029 - accuracy: 0.7879

Test Loss 1.4028648138, Test Accuracy 0.787878811359


In [52]:
#infer labels on a batch or a dataset of batches
predictions = model.predict(test_data)

for prediction, smoke_detector in zip(predictions[:66], list(test_data)[0][1][:66]):
    print("Smoke Detector: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("Smoke Detector" if bool(smoke_detector) else "No Smoke Detector"))
  


Smoke Detector: 61.22%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.01%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.05%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.93%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.12%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.00%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.93%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.02%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.00%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.94%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.00%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.96%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.00%  | Actual outcome:  No Smoke Detector
Smoke Detector: 99.96%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.04%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.03%  | Actual outcome:  No Smoke Detector
Smoke Detector: 0.02%  | Actual ou

In [53]:
#save predictions to list
li=(predictions)[:66].tolist()

In [54]:
#create list with predictions and parameters from test data
print("Fire Sprinkler only")
new_li=[]
count=0
temp_li=[]
for i in list(test_data)[0][0]['Fire_Sprinkler']:
    temp_li.append(format(i))
    new_li.append(temp_li)
    temp_li=[]
    count+=1
print(new_li)
print("FS and Building Occupancy")
count=0
for i in list(test_data)[0][0]['Building_Occupancy']:
    new_li[count].append(format(i))
    temp_li=[]
    count+=1
print(new_li)
print("FS BS and Room Name")
count=0
for i in list(test_data)[0][0]['Room_Name']:
    new_li[count].append(format(i))
    temp_li=[]
    count+=1
print(new_li)

print("FS BS RM and Room Area")
count=0
for i in list(test_data)[0][0]['numeric']:
    new_li[count].append(float(format(i[0])))
    temp_li=[]
    count+=1
print(new_li)

print("FS BS RM RA and Prediction")
count=0
for i in li:
    if i[0]<0.5:i[0]=0
    else: i[0]=1
    new_li[count].append(i[0])
    temp_li=[]
    count+=1
print(new_li)

Fire Sprinkler only
[['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes'], ['Yes']]
FS and Building Occupancy
[['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical Building'], ['Yes', 'Medical 

In [55]:
#create pandas dataframe with columns using list and transfer to csv file
CSV_COLUMNS = ['Fire_Sprinkler', 'Building_Occupancy', 'Room_Name', 'Room_Area', 'Smoke_Detector']
new_df=pd.DataFrame(new_li,columns=CSV_COLUMNS)
print(new_df)
new_df.to_csv(r'data-pred.csv')

   Fire_Sprinkler Building_Occupancy        Room_Name    Room_Area  \
0             Yes   Medical Building        Reception   322.150787   
1             Yes   Medical Building          Storage   386.375000   
2             Yes   Medical Building   Procedure Room   226.376236   
3             Yes   Medical Building     Control Room   183.625000   
4             Yes   Medical Building   Generator Room   155.375000   
5             Yes   Medical Building       Break Room   174.208328   
6             Yes   Medical Building     Control Room   189.273895   
7             Yes   Medical Building    Nurse Station   304.737244   
8             Yes   Medical Building          Kitchen   690.791687   
9             Yes   Medical Building         Corridor  2883.666748   
10            Yes   Medical Building         Bathroom   172.798615   
11            Yes   Medical Building   Elevator Lobby   513.215271   
12            Yes   Medical Building      Locker Room   229.625000   
13            Yes   